In [1]:
import os
from pathlib import Path

try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import userdata

    repo_name = "dgcnz/dl2"
    url = f"https://{userdata.get('gh_pat')}@github.com/{repo_name}.git"
    !git clone --branch meta-issue-alpha-reproduction- {url}
    print("\nCurrent Directory:")
    %cd dl2
    #!pip install torch torchvision numpy matplotlib git+https://github.com/AMLab-Amsterdam/lie_learn escnn scipy
    !pip install torchvision git+https://github.com/AMLab-Amsterdam/lie_learn escnn lightning wandb git+https://github.com/dgcnz/neuralyze git+https://github.com/QUVA-Lab/e2cnn
    !pip install -r requirements.txt


else:  # automatically checks if the current directory is 'repo name'
    curdir = Path.cwd()
    print("Current Directory", curdir)
    assert (
        curdir.name == "dl2" or curdir.parent.name == "dl2"
    ), "Notebook cwd has to be on the project root"
    if curdir.name == "notebooks":
        %cd ..

Cloning into 'dl2'...
remote: Enumerating objects: 1215, done.
remote: Counting objects: 100% (1147/1147), done.
remote: Compressing objects: 100% (395/395), done.
remote: Total 1215 (delta 729), reused 1107 (delta 711), pack-reused 68
Receiving objects: 100% (1215/1215), 62.19 MiB | 32.31 MiB/s, done.
Resolving deltas: 100% (730/730), done.

Current Directory:
/content/dl2
  Cloning https://github.com/AMLab-Amsterdam/lie_learn to /tmp/pip-req-build-ykdm4dap
  Running command git clone --filter=blob:none --quiet https://github.com/AMLab-Amsterdam/lie_learn /tmp/pip-req-build-ykdm4dap
  Resolved https://github.com/AMLab-Amsterdam/lie_learn to commit 1ccc2106e402d517a29de5438c9367c959e67338
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/dgcnz/neuralyze to /tmp/pip-req-build-lpgpvp9h
  Running command git clone --filter=blob:none -

In [7]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 k

In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

In [3]:
path_dict ={'cnn_v2': "uva-dl2/CNN_RotMNIST/model-4l530hst:v2",
            'cnn_v24': "uva-dl2/CNN_RotMNIST/model-4l530hst:v24",
            'scnn_v2': "uva-dl2/C8steerable_RotMNIST/model-u3vh8ed9:v2",
            'scnn_v24': "uva-dl2/C8steerable_RotMNIST/model-u3vh8ed9:v24"

}
dict_string = json.dumps(path_dict)

In [18]:
!python -m src.compute_measures --weight_decay 1e-5 --percentage_data 20 --ckpt_path_dict dict_string --equivariance_error True

usage: compute_measures.py [--help] [--hydra-help] [--version] [--cfg {job,hydra,all}] [--resolve]
                           [--package PACKAGE] [--run] [--multirun] [--shell-completion]
                           [--config-path CONFIG_PATH] [--config-name CONFIG_NAME]
                           [--config-dir CONFIG_DIR] [--experimental-rerun EXPERIMENTAL_RERUN]
                           [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
                           [overrides ...]
compute_measures.py: error: unrecognized arguments: --weight_decay --percentage_data 20 --ckpt_path_dict dict_string --equivariance_error True


In [ ]:
with open('logs/hessian_values/spectra.json') as json_file:
    metric_dict = json.load(json_file)

##Hessian Spectra

In [ ]:
def plot_fn(spectrums: list, names: list, epoch: int):
    fig, ax = plt.subplots()
    # Create kernel density estimate
    for spectrum, name in zip(spectrums, names):
        kde = gaussian_kde(spectrum)

        # Create x values for the plot
        x = np.linspace(min(spectrum), max(spectrum), 1000)

        # Compute the density estimate
        density = kde(x)

        # Plot density estimate as a line
        ax.plot(x, density, label=name)

    # Add labels and title
    ax.set_xlabel("Max Eigenvalue")
    ax.set_ylabel("Density")
    ax.set_title("Density plot of max eigenvalues at epoch " + str(epoch))
    ax.legend()
    plt.show()

##Sharpeness


In [ ]:


#and then plot the ones you want to plot together.

##Equivariance learned/error